# Analysis of Processed Experimental Data

This notebook is a resource for comparing multiple replicates of the same tumor-tcell simulation and multiple conditions against each other with statistics

In [ ]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl

#Analysis tumor-tcell modules needed
from tumor_tcell.library.population_plots import death_group_plot
from tumor_tcell.library.population_plots import population_group_plot

The imput variables are:
* experiment_dir - these are the directories that contain the replicates of the simulations
* experiment_name - this is what the analysis folder will be called

In [ ]:
#Input variables
experiment_dir = ['25% PD1+/','75% PD1+/']
experiment_name = 'PD1_25vs75'

Default states and dictionary that do not need to be changed usually

In [ ]:
#Save variables in looping dictionary order for correcting later
conditions_list = [i.split("/", 1)[0] for i in experiment_dir]

#Experiment number
exp_dict = {i: conditions_list[i] for i in range(0, len(conditions_list), 1)}

#Cell types
cell_dict = {0:'Tumor', 1:'T Cell'}

#Cell States
cell_states_tcell = ['PD1n','PD1p']
cell_states_tumor = ['PDL1n','PDL1p']
cell_states = [cell_states_tumor,cell_states_tcell]

analysis_dir and save_dir can be changed based on local directory structure. 

This set of code extracts all relevant data and makes a list of dataframes plot_data_list and death_data_list for further processing.

In [ ]:
#Get csv saved in experiment id library
analysis_dir = '/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/'
save_dir = analysis_dir+'Multiple_analysis/'

#Make new output analysis folder with Experiment Name
analysis_out_dir = save_dir + experiment_name 
os.makedirs(analysis_out_dir, exist_ok=True)

#make into a list for looping
analysis_dir_list=[]
for exp in experiment_dir:
    add_dir = analysis_dir+exp
    analysis_dir_list.append(add_dir)

#Create a list of dataframes for concatenation
plot_data_list = []  
death_data_list = []  

#Loop through directories containing replicates of experimental condition
for exp_dir in analysis_dir_list:
    os.chdir(exp_dir)
    experiment_list = next(os.walk('.'))[1]
    
    plot_list = []
    death_list = []
    df_tumor_death_list = []
    df_tcell_death_list = []
    tumor_plot_list = []
    tcell_plot_list = []
    
    #Loop through experiments
    for experiment in experiment_list:
        experiment_directory = exp_dir+experiment
        os.chdir(experiment_directory)
        
        #Import all the plot and death files into a list of dataframes
        file_list = os.listdir()
        #For control conditions that do not have t cells need if statement
        if 'tcell_plot.csv' in file_list:
            df_tumor_death = pd.read_csv('tumor_death.csv')
            df_tumor_death_list.append(df_tumor_death)

            tumor_plot = pd.read_csv('tumor_plot.csv')
            tumor_plot_list.append(tumor_plot)

            df_tcell_death = pd.read_csv('tcell_death.csv')
            df_tcell_death_list.append(df_tcell_death)

            tcell_plot = pd.read_csv('tcell_plot.csv')
            tcell_plot_list.append(tcell_plot)
            
        else:
            df_tumor_death = pd.read_csv('tumor_death.csv')
            df_tumor_death_list.append(df_tumor_death)

            tumor_plot = pd.read_csv('tumor_plot.csv')
            tumor_plot_list.append(tumor_plot)
    
    #Make 2 lists for different cell types
    plot_list.extend([tumor_plot_list,tcell_plot_list])
    death_list.extend([df_tumor_death_list,df_tcell_death_list])
    
    #Make a new upper list based on experimental condition and do one for cells and death data
    plot_data_list.append(plot_list)
    death_data_list.append(death_list)

plot_data_list[0]    

This next set of code loops through the dataframes to perform statistics on the replicates and generates a list of dataframes cell_plot_list as output based on the number of experiments to compare

In [ ]:
cell_plot_list = []
#loop through experiments
for exp in range(len(plot_data_list)):
    df_list_plot = []
    exp_df = plot_data_list[exp]
    #loop through cells
    for cell in range(len(exp_df)):
        cell_state_list_1 = []
        cell_state_list_2 = []
        total_list = []
        cell_list = []
        cell_df = exp_df[cell]
        
        #need condition for control samples
        if len(cell_df)==0:
            continue
            
        else:
            #Count the unique cells and individual cell states for each experiment
            for i in range(len(cell_df)):
                #get unique cells
                total_cell = cell_df[i].groupby(['time','experiment_id'])['cell'].nunique().reset_index()
                total_list.append(total_cell)

                cell_state_df = cell_df[i].groupby(['time', 'cell_state','experiment_id'])['cell'].nunique().reset_index()
                state_1 = cell_state_df.loc[cell_state_df['cell_state'] == cell_states[cell][0]]
                cell_state_list_1.append(state_1)

                state_2 = cell_state_df.loc[cell_state_df['cell_state'] == cell_states[cell][1]]
                cell_state_list_2.append(state_2)

            df_total = pd.concat(total_list)
            df_state_1 = pd.concat(cell_state_list_1)
            df_state_2 = pd.concat(cell_state_list_2)
        
        #Get the mean and sem for each condition
        df_total_ave = df_total.groupby(['time']).agg({'cell': ['mean', 'sem']})
        df_total_ave.columns = df_total_ave.columns.droplevel(0)
        df_total_ave = df_total_ave.rename_axis(None, axis=1)
        df_total_ave.reset_index(inplace=True)
        
        
        df_state_1_ave = df_state_1.groupby(['time','cell_state']).agg({'cell': ['mean', 'sem']})
        df_state_1_ave.columns = df_state_1_ave.columns.droplevel(0)
        df_state_1_ave = df_state_1_ave.rename_axis(None, axis=1)
        df_state_1_ave.reset_index(inplace=True)
    
        df_state_2_ave = df_state_2.groupby(['time','cell_state']).agg({'cell': ['mean', 'sem']})
        df_state_2_ave.columns = df_state_2_ave.columns.droplevel(0)
        df_state_2_ave = df_state_2_ave.rename_axis(None, axis=1)
        df_state_2_ave.reset_index(inplace=True)
        
        cell_state_all = pd.concat([df_state_1_ave, df_state_2_ave])
        #save both the total and cell_state df in list
        cell_list.append(df_total_ave)
        cell_list.append(cell_state_all)
        #save in another list based on cells
        df_list_plot.append(cell_list)
    #save in another list based on experiment
    cell_plot_list.append(df_list_plot)
        
cell_plot_list

This set of code produces the plot that shows the mean and sem for the various population plots

In [ ]:
#loop first based on the cell type
for cell in range(len(cell_plot_list[0])):
    
    SMALL_SIZE = 18
    MEDIUM_SIZE = 22
    BIGGER_SIZE = 24

    pl.rc('font', size=SMALL_SIZE)  # controls default text sizes
    pl.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title
    pl.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
    pl.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    pl.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    pl.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
    pl.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    # Create plot
    pl.figure(figsize=(8, 4))

    #loop next based on experiment
    for exp in range(len(cell_plot_list)): 
        
        #need for control condition
        if len(cell_plot_list[exp])==1 & cell>0:
            continue
        #Create plot for each experiment per cell type
        else:
            df_plot = cell_plot_list[exp][cell][0]

            # create vectors to use for SEM plotting
            M_new_vec = np.array(df_plot['mean'])
            Sigma_new_vec = np.array(df_plot['sem'])
            lower_bound = M_new_vec - Sigma_new_vec
            upper_bound = M_new_vec + Sigma_new_vec

            # Create plot
            ttl_1 = sns.lineplot(data=df_plot, x="time", y='mean', label=exp_dict[exp])
            pl.title("Total " + str(cell_dict[cell]))
            pl.ylabel('# of cells')
            pl.fill_between(df_plot['time'], lower_bound, upper_bound, alpha=.3)
            pl.legend(title="Experiment")
            pl.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


    pl.savefig(analysis_out_dir + '/' + cell_dict[cell] + '_cell_total.png', transparent=True, format='png',
               bbox_inches='tight', dpi=300)
        
    # Create plot
    pl.figure(figsize=(8, 4))

    for exp in range(len(cell_plot_list)):    
        if len(cell_plot_list[exp])==1 & cell>0:
            continue
        else:
            df_plot = cell_plot_list[exp][cell][1]

            #loop through each state to loook at substates of the cells
            for state in df_plot.cell_state.unique():
                # create vectors to use for SEM plotting
                df_state = df_plot[df_plot.cell_state==state]
                M_new_vec = np.array(df_state['mean'])
                Sigma_new_vec = np.array(df_state['sem'])
                lower_bound = M_new_vec - Sigma_new_vec
                upper_bound = M_new_vec + Sigma_new_vec

                # Create plot
                ttl_2 = sns.lineplot(data=df_state, x="time", y='mean',label=exp_dict[exp]+' '+state)
                pl.title("Total Subtype of " + cell_dict[cell])
                pl.ylabel('# of cells')
                pl.fill_between(df_state['time'], lower_bound, upper_bound, alpha=.3)
                pl.legend(title="Experiment")
                pl.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    pl.savefig(analysis_out_dir + '/' + cell_dict[cell] + '_cell_subtypes.png', transparent=True, format='png',
               bbox_inches='tight', dpi=300)

This next set of code loops through the dataframes to perform statistics on the replicates and generates a list of dataframes death_plot_list as output based on the number of experiments to compare death statistics

In [ ]:
death_plot_list = []
#loop through experiments
for exp in range(len(death_data_list)):
    death_list_plot = []
    exp_df = death_data_list[exp]
    
    #loop through cell types
    for cell in range(len(exp_df)):
        death_cell_list = []
        death_plot_l = []
        other_death_plot_l =[]
        cell_df = exp_df[cell]
        
        #need for control condition (no T cells)
        if len(cell_df)==0:
            continue
            
        else:
        
            for i in range(0,len(cell_df)):    
                #convert the time back to seconds
                cell_df[i]['time_s'] = cell_df[i].time*3600
                cell_df[i].time_s = cell_df[i].time_s.round(-1).astype(int)
                #list of unique death types
                death_types_list = list(cell_df[i]['death'].unique())
                #create a dictionary of death types and map to sum for .agg function
                dict_death = {}
                for death_type in death_types_list:
                    dict_death[death_type] = 'sum'
                #add exp_id to dict dataframe
                dict_death['experiment_id'] = 'first'
                #aggregate to combine multiple cell deaths at the same time point
                death_agg = cell_df[i].groupby('time_s').agg(dict_death)
                #Add in missing intervals (no death)
                death_agg_time = death_agg.reindex(np.arange(0, cell_df[i].time_s.max()+60,60),fill_value=0)
                #change to timeindex so that we can resample the data for better mean and sem calculations by the hour
                death_agg_time.index = pd.to_timedelta(death_agg_time.index, unit='s')
                df_hour = death_agg_time.resample('h').sum()
                for death_type in death_types_list:
                    df_hour['total_' + str(death_type)] = df_hour[death_type].cumsum()
                total_col_t = [col for col in df_hour.columns if 'total' in col]
                df_hour['total_death'] = df_hour[total_col_t].sum(axis=1)
                df_hour['time'] = (df_hour.index.days*24+df_hour.index.seconds/3600)

                #create dataframes for the plotting
                total_col = [col for col in df_hour.columns if 'total' in col]
                death_plot = pd.melt(df_hour, id_vars=[ 'time',], value_vars=total_col)
                death_plot.rename(columns={'variable': 'death type', 'value': 'death count'}, inplace=True)
                death_plot['experiment_id'] = i

                total_death_plot = death_plot.loc[death_plot['death type'] == 'total_death']
                other_death_plot = death_plot.loc[~(death_plot['death type'] == 'total_death')]

                death_plot_l.append(total_death_plot)
                other_death_plot_l.append(other_death_plot)

        # Concatenate all
        death_plots = pd.concat(death_plot_l)
        other_death_plots = pd.concat(other_death_plot_l)

        # Get mean and sem for each
        total_death_plot_ave = death_plots.groupby('time').agg({'death count': ['mean', 'sem']})
        total_death_plot_ave.columns = total_death_plot_ave.columns.droplevel(0)
        total_death_plot_ave = total_death_plot_ave.rename_axis(None, axis=1)
        total_death_plot_ave.reset_index(inplace=True)
        
        other_death_plot_ave = other_death_plots.groupby(['time','death type']).agg({'death count': ['mean', 'sem']})
        other_death_plot_ave.columns = other_death_plot_ave.columns.droplevel(0)
        other_death_plot_ave = other_death_plot_ave.rename_axis(None, axis=1)
        other_death_plot_ave.reset_index(inplace=True)

        death_cell_list.append(total_death_plot_ave)
        death_cell_list.append(other_death_plot_ave)
        death_list_plot.append(death_cell_list)
    
    death_plot_list.append(death_list_plot)
        
death_plot_list

This set of code produces the plot that shows the mean and sem for the various population plots

In [ ]:
#First loop through cell types
for cell in range(len(death_plot_list[0])):

    SMALL_SIZE = 18
    MEDIUM_SIZE = 22
    BIGGER_SIZE = 24

    pl.rc('font', size=SMALL_SIZE)  # controls default text sizes
    pl.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title
    pl.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
    pl.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    pl.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    pl.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
    pl.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    # Create plot
    pl.figure(figsize=(8, 4))

    #Loop through experiments and have condition for control that does not have T cells
    for exp in range(len(death_plot_list)):    
        if len(death_plot_list[exp])==1 & cell>0:
            continue
        else:
        
            df_plot = death_plot_list[exp][cell][0]

            # create vectors to use for SEM plotting
            M_new_vec = np.array(df_plot['mean'])
            Sigma_new_vec = np.array(df_plot['sem'])
            lower_bound = M_new_vec - Sigma_new_vec
            upper_bound = M_new_vec + Sigma_new_vec

            # Create plot
            ttl_1 = sns.lineplot(data=df_plot, x="time", y='mean', label=exp_dict[exp])
            pl.title("Total " + cell_dict[cell])
            pl.ylabel('# of deaths')
            pl.fill_between(df_plot['time'], lower_bound, upper_bound, alpha=.3)
            pl.legend(title="Experiment")
            pl.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    pl.savefig(analysis_out_dir + '/' + cell_dict[cell]+'_death_total.png', transparent=True, format='png',
               bbox_inches='tight', dpi=300)

    # Create plot
    pl.figure(figsize=(8, 4))

    for exp in range(len(death_plot_list)):    
        if len(death_plot_list[exp])==1 & cell>0:
            continue
        else:
            df_plot = death_plot_list[exp][cell][1]

            for death_type in df_plot['death type'].unique():
                # create vectors to use for SEM plotting
                df_state = df_plot[df_plot['death type']==death_type]
                M_new_vec = np.array(df_state['mean'])
                Sigma_new_vec = np.array(df_state['sem'])
                lower_bound = M_new_vec - Sigma_new_vec
                upper_bound = M_new_vec + Sigma_new_vec

                # Create plot
                ttl_2 = sns.lineplot(data=df_state, x="time", y='mean',label=exp_dict[exp]+' '+death_type)
                pl.title("Total Subtype of " + cell_dict[cell])
                pl.ylabel('# of deaths')
                pl.fill_between(df_state['time'], lower_bound, upper_bound, alpha=.3)
                pl.legend(title="Experiment")
                pl.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    pl.savefig(analysis_out_dir + '/' + cell_dict[cell] + '_death_subtype.png', transparent=True, format='png',
               bbox_inches='tight', dpi=300)       

This set of code generates output for comparison of averages to in vitro or in vivo based experiments

In [ ]:
#### Create output for the cell percentages at the end of the simulation to compare with experimental
df_sub_list = []
for cell in range(len(cell_plot_list[1])):

    for exp in range(len(cell_plot_list)): 
        
        if len(cell_plot_list[exp])==1 & cell>0:
            continue
        else:
            dfplot = cell_plot_list[exp][cell][1]
            for state in dfplot.cell_state.unique():
                df_state = dfplot[dfplot.cell_state==state]
                #Get last cell in the dataframe 
                dfsub = df_state.iloc[-1:].copy()
                dfsub['experiment'] = exp_dict[exp]
                dfsub['cell'] = cell_dict[cell]
                df_sub_list.append(dfsub)

df_final_cell_counts = pd.concat(df_sub_list)
df_final_cell_counts.to_csv(analysis_out_dir+'/final_cell_counts.csv')